In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
np.random.seed(1000)
import matplotlib.pyplot as plt
%matplotlib inline
import os
from snorkel.learning import GenerativeModel
from scipy import sparse
import matplotlib.pyplot as plt
from functools import partial
from sklearn.metrics import f1_score
from scipy.optimize import minimize
from sklearn.utils import shuffle

/anaconda2/envs/snorkel/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
# simulated data 
#(1,1,1), (-1,-1,-1), (1, 0.5, 0.2)
mean_1 = [1,1,1]
mean_0 = [-1, -1, -1]
std = [1, 0.5, 0.2]
#std = [0.1, 0.1, 0.1]
n = 500
X_1 = np.random.normal(mean_1, std, size=(n, len(mean_1)))
X_0 = np.random.normal(mean_0, std, size=(n,len(mean_0)))
X = np.concatenate([X_1, X_0], axis=0)
Y = np.zeros((2*n,))
Y[0:n,] = 1
Y[n:,] = -1 
tmp = shuffle(range(2*n))
X = X[tmp,:]
Y = Y[tmp,]
chosen_data = range(2*n)

n = 250
X_1_test = np.random.normal(mean_1, std, size=(n, len(mean_1)))
X_0_test = np.random.normal(mean_0, std, size=(n,len(mean_0)))
Y_test = np.zeros((2*n,))
Y_test[0:n,] = 1
Y_test[n:,] = -1 
X_test = np.concatenate([X_1_test, X_0_test], axis=0)







In [3]:
def f(alpha, L_fns=None, ind=None, weights=None, epochs=None):
    """L_fns: list of functions
       l: list of indicator for whether thresholds involve in the particular function 
    """
    #alpha = np.exp(alpha)
    for i in range(len(ind)):
        L_fns[ind[i]] = partial(L_fns[ind[i]], thre=alpha[i])
        
    L = np.zeros((len(L_fns),len(X))).astype(int)
    
    for j in range(len(chosen_data)):
        i = chosen_data[j]
        for i in range(len(L_fns)):
            L[i,j] = L_fns[i](X[j])
#         L[0,j] = L_fns[0](p_keys['has_human'][i], p_keys['has_road'][i])
#         L[1,j] = L_fns[1](p_keys['has_human'][i], p_keys['has_cars'][i])
#         L[2,j] = L_fns[2](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_distance'][i])
#         L[3,j] = L_fns[3](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_size'][i])
#         L[4,j] = L_fns[4](p_keys['has_human'][i], p_keys['has_bike'][i], p_keys['bike_human_num'][i])
        
    L_train = sparse.csr_matrix(L.T)
    gen_model = GenerativeModel()
    gen_model.train(L.T, epochs=epochs, decay=0.95, step_size= 0.01/ L.shape[1], reg_param=1e-6)
    if weights is not None:
        gen_model.weights = weights
    train_marginals, likelihood = gen_model.marginals(L_train)
    labels = 2 * (train_marginals > 0.5) - 1
    gen = np.mean(labels == Y)
    f1_gen = f1_score(Y, labels)
    if weights is None:
        return (gen,f1_gen,labels), gen_model.weights, -likelihood 
    else:
        return -likelihood 

    
    

In [4]:
def LF_1(x):
    if x[0] < -0.5:
        return -1
    elif x[0] > 0.5:
        return 1 
    else:
        return 0  

def LF_1(x, thre):
    if x[0] < -thre:
        return -1
    elif x[0] > thre:
        return 1 
    else:
        return 0  

def LF_2(x, thre):
    if x[1] < -thre:
        return -1 
    elif x[1] > thre:
        return 1 
    else:
        return 0 

def LF_3(x, thre):
    if x[2] < -thre:
        return -1 
    elif x[2] > thre:
        return 1 
    else:
        return 0 
        

In [7]:
L_fns = [LF_1, LF_2, LF_3]
ind = [0, 1, 2]
iters = 5
alpha = [2, 2, 2]
# init = np.zeros((3,2))
# init[:,0] = [4, 2, 1]
# init[:,1] = [4, 2, 1]

for i in range(iters):
    ans, weights, fvalue = f(alpha, L_fns=L_fns, ind=ind, epochs=200)
    print(fvalue)
    f_new = partial(f, L_fns=L_fns, ind=ind, epochs=0, weights=weights)
    alpha = minimize(f_new, alpha, method='Nelder-Mead').x
    print(f_new(alpha=alpha))
    print(alpha)
    print(ans[0:2]) 


    


-751.002768191
-1008.33594702
[-0.00486206  2.22979729  2.59256973]
(0.57699999999999996, 0.26943005181347146)
-1189.21041712
-1667.09689405
[-0.00627649 -0.01115578  3.10980531]
(0.82599999999999996, 0.82634730538922152)
-1882.74319656
-1882.74319656
[-0.00627649 -0.01115578  3.10980531]
(0.97799999999999998, 0.97795591182364716)
-1882.74319656
-1882.74319656
[-0.00627649 -0.01115578  3.10980531]
(0.97799999999999998, 0.97795591182364716)
-1882.74319656
-1882.74319656
[-0.00627649 -0.01115578  3.10980531]
(0.97799999999999998, 0.97795591182364716)


In [14]:
# simulations compare: [0, 1] random vs [0, 1] learning threshold 
sim_n = 1
acc = [[0,0] for _ in range(sim_n)]
acc_baseline = [[0,0] for _ in range(sim_n)]
f1 = [[0,0] for _ in range(sim_n)]
f1_baseline = [[0,0] for _ in range(sim_n)]
lower = 5
upper = 5
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def metric(labels):
    clf = LinearDiscriminantAnalysis()
    clf.fit(X, labels)
    labels = clf.predict(X_test)
    gen = np.mean(labels == Y_test)
    f1_gen = f1_score(Y_test, labels)
    return gen, f1_gen 
tmp = np.random.uniform(lower, upper, (len(ind),sim_n))


In [15]:
print(X.shape, X_test.shape, Y.shape, Y_test.shape)

(1000, 3) (500, 3) (1000,) (500,)


In [16]:
# baseline
for i in range(sim_n):
    tmp_alpha = tmp[:,i]
    ans, _, _ = f(tmp_alpha, L_fns=L_fns, ind=ind, epochs=1000)
    acc_baseline[i][0], f1_baseline[i][0] = ans[0:2]
    labels = ans[2]
    acc_baseline[i][1], f1_baseline[i][1] = metric(labels)
print(np.mean([x[1] for x in acc_baseline]), np.mean([x[1] for x in f1_baseline]))
        

0.5 0.0


/anaconda2/envs/snorkel/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda2/envs/snorkel/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
/anaconda2/envs/snorkel/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
# our method, extract max 
for i in range(sim_n):
    alpha = tmp[:,i]
    for j in range(iters):
        ans, weights, _ = f(alpha, L_fns=L_fns, ind=ind, epochs=200)
        f_new = partial(f, L_fns=L_fns, ind=ind, epochs=0, weights=weights)
        alpha = minimize(f_new, alpha, method='Nelder-Mead').x
    print(i,j)
    acc[i][0], f1[i][0] = ans[0:2]
    labels = ans[2]
    acc[i][1], f1[i][1] = metric(labels)
print(np.mean([x[1] for x in acc]), np.mean([x[1] for x in f1]))  
        

KeyboardInterrupt: 

In [10]:
print(tmp)

[[ 1.85732701  0.82346156  0.09922926  1.88876315  0.22761698  0.08153212
   0.73529025  0.09201986  0.80629535  0.7846683   0.83422968  1.45311759
   1.95127227  0.41294227  1.66505524  0.65916574  1.56140248  1.11103006
   1.03209949  1.75709669  0.48289837  1.85754753  0.13199199  1.38453394
   1.75728017  0.73654868  0.63808297  0.5908707   0.48764574  1.91803055
   1.60001319  0.4405304   1.16103229  1.73974143  1.51382624  0.1661766
   1.15220462  0.99681015  1.18853847  0.06297392  0.75084641  1.73512788
   1.65659983  0.42574965  0.06336282  1.67705328  0.8422862   1.53372225
   1.12728429  1.11852553]
 [ 0.91072865  0.62899112  0.61083202  0.98265218  1.40624206  0.64551136
   0.28644333  0.10985214  0.47016806  1.35518096  1.17086215  0.98175003
   1.64996595  1.9866008   0.43375988  0.30896648  0.35885572  0.79464605
   1.33363345  1.68281421  1.43321541  0.41357517  1.85386714  0.3317915
   1.61196313  1.92673448  0.25041791  0.38242737  1.1398077   1.22408875
   1.52547257

In [11]:
print(np.mean([x[0] for x in acc_baseline]), np.mean([x[0] for x in f1_baseline]))

print(np.mean([x[0] for x in acc]), np.mean([x[0] for x in f1]))  
        

0.91336 0.895549614307
0.98984 0.989765067912
